In [27]:
%matplotlib inline
from PIL import Image
import numpy
import os
import re
import random
from resizeimage import resizeimage
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.decomposition import PCA
import copy
import matplotlib.image as mpimg
from sklearn.decomposition import RandomizedPCA

In [59]:
# resize images
# resizeImages('../data/yalefaces/', 'subject01')
def resizeImagesToDest(source, dest, prefix, height, noiseAlgo=None):    
    for root, dirs, files in os.walk(source):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(os.path.join(source, name))
                (origWidth, origHeight) = img.size
                if (height == origHeight):
                    resizedImg = img
                else:
                    resizedImg = resizeimage.resize_height(img, height)
                    
                (width, height) = resizedImg.size
                if noiseAlgo:
                    noisyImg = noiseAlgo(numpy.asarray(resizedImg))
                    Image.fromarray(numpy.uint8(noisyImg.reshape(height,width))).save(dest+name)
                else:
                    resizedImg.save(dest+name)

def resizeImages(source, prefix):
    resizeImagesToDest(source, 'yalefaces_resized/', prefix, 50)
                
# images into array: http://stackoverflow.com/questions/13550376/pil-image-to-array-numpy-array-to-array-python
def readImagesIntoMatrix(path, prefix):
    images = None
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)                
                (width, height) = img.size
                imgArr = numpy.asarray(img).reshape(height * width)
                if images is not None:
                    images = numpy.vstack([images, imgArr])
                else:
                    images = imgArr
    return images

def saveImages(path, images, width, height):
    for i in range(0, len(images)):
        Image.fromarray(numpy.uint8(images[i]).reshape(height, width)).save(path+str(i)+'.gif')
        
def addGaussianNoise(image):
    noisyImage = copy.deepcopy(image)
    mu = noisyImage.mean()
    std = noisyImage.std()
    noise = 0.2 * numpy.random.normal(mu, std, noisyImage.shape)
    for i in range(len(image)):
        for j in range(len(image[0])):
            val = image[i][j]
            if val + noise[i][j] >= 255:
                noisyImage[i][j] = 255
            elif val + noise[i][j] <= 0:
                noisyImage[i][j] = 0
            else:
                noisyImage[i][j] = val + noise[i][j] 
    
    return noisyImage  
    
def addSparseNoise(image):
    noisyImage = copy.deepcopy(image)
    for i in range(len(image)):
        for j in range(len(image[0])):
            rand = random.random()
            if (rand <= 0.01):
                noisyImage[i][j] = 0
    
    return noisyImage

In [62]:
resizeImages('yalefaces/', 'yale.*')
imageList = readImagesIntoMatrix('yalefaces_resized/', '.*').T

print("Images %s" % imageList)

Image.fromarray(numpy.uint8(imageList.T[0].reshape(50,44))).save("secondyaletest/testsaveimage1.gif")

Images [[155   4 135 ..., 161  84   5]
 [152   4 135 ..., 162  84   5]
 [136   4 120 ..., 166  88   5]
 ..., 
 [ 11 145   6 ...,   6   4 163]
 [ 11 141   6 ...,   6   6 178]
 [ 12 136   7 ...,  13   8 187]]


In [64]:
(width, height) = 44, 50
# average Image
#print('Num images %s' % (imageList.shape,))
avgImg = imageList.mean(axis=1)
print('Average image %s' % (avgImg))
normalizedImageList = imageList - avgImg.reshape(width*height,1)*numpy.ones((1,2414))
Image.fromarray(numpy.uint8(avgImg.reshape(height,width))).save('secondyaletest/avgImg.gif')

Average image [ 59.02526926  62.63835957  64.9908865  ...,  49.28583264  46.52734051
  40.11267606]


In [68]:
def normalize_face(vector):
    minValue = numpy.amin(vector)
    maxValue = numpy.amax(vector)
    scale = (255./(maxValue-minValue))
    return (vector - minValue)*scale

# Fit with PCA for 90% of variance
pca = PCA(0.9).fit_transform(normalizedImageList)

for i in range(len(pca.T)):
    eigv = pca[:,i].reshape(1,width * height).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(height,width))).save('secondyaletest/eigenface%s.gif' % i)

IndexError: index 33 is out of bounds for axis 1 with size 33

In [39]:
def top_k_eigenface(image, eigenVectors, k):
    start_vector = numpy.zeros((50*66))
    for i in range(0, k-1):
        eigv = eigenVec[:,i].reshape(3300)
        weight = numpy.inner(image, eigv)
        start_vector = start_vector + weight * eigv
    return start_vector

numComponents = 15
projections = []
for i in range(0, len(imagesDiff.T)):
    projection = top_k_eigenface(imagesDiff.T[i].reshape(3300), eigv, numComponents)+avgImg
    projections.append(projection)
    #print ('Projection dimensions %s' % (projection.shape,))
    #Image.fromarray(numpy.uint8(projection.reshape(50,66))).save('test/happy_13_projection_of_subject%s.gif' % i)
    projection = top_k_eigenface(imagesDiff.T[i].reshape(3300), eigv, 3300)+avgImg
    #Image.fromarray(numpy.uint8(projection.reshape(50,66))).save('test/happy_3300_projection_of_subject%s.gif' % i)